## Visualization #3

    Is the distribution of students in NCWIT data set similar to the population for ACT scores?

In [9]:
import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, widgetbox, layout
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import Select
from ipywidgets import interact
from scipy.stats import norm
from IPython.display import display

output_notebook()
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999

Loading BokehJS ...

In [10]:
ncwit_df = pd.read_csv("NCWIT-Data Cleaned.csv", low_memory = False)

ACT_df = pd.read_csv("ACT Data.csv")
ACT_df = ACT_df.loc[ACT_df.Year > 2005]
'''
ncwitMod = ncwit_df.loc[ncwit_df["School Year"] == "2007-2008"].copy()
fVal, fCounts = np.unique(ncwitMod["Enroll, Female: Avg. ACT Math Score (Enrl F)"].round(0).dropna(), return_counts=True)
fCounts = fCounts/np.sum(fCounts)
mVal, mCounts = np.unique(ncwitMod["Enroll, Male: Avg. ACT Math Score (Enrl M)"].round(0).dropna(), return_counts=True)
mCounts = mCounts/np.sum(mCounts)
fMean = ACT_df.Female[ACT_df.Year == 2007]
mMean = ACT_df.Male[ACT_df.Year == 2007]
stdDev = ACT_df["Standard Deviation"][ACT_df.Year == 2007]
'''
# The function is supposed to allow the notebook to update values in the graphs.
# It works, sometimes. See two cells below for where it's called.
def update(Year):
    headerYear = Year
    # Reference year of records to pull.
    ncwitYear = str(Year)+"-"+str(int(Year)+1)
    print(ncwitYear)
    # Create temporary dataframes for NCWIT data.
    ncwitMod = ncwit_df.loc[ncwit_df["School Year"] == ncwitYear].copy()
    # Generate values for graphs below.
    fMean = ACT_df.Female[ACT_df.Year == int(Year)]
    mMean = ACT_df.Male[ACT_df.Year == int(Year)]
    stdDev = ACT_df["Standard Deviation"][ACT_df.Year == int(Year)]
    # Round values to nearest integer, drop null values and return counts for each value.
    # https://stackoverflow.com/questions/12207326/frequency-table-for-a-single-variable
    # Code finds the # of unique values for ACT scores (after rounding and dropping nan values) and
    # returns the count of each observation of each value
    fVal, fCounts = np.unique(ncwitMod["Enroll, Female: Avg. ACT Math Score (Enrl F)"].round(0).dropna(), return_counts=True)
    # Normalize counts.
    fCounts = fCounts/np.sum(fCounts)
    mVal, mCounts = np.unique(ncwitMod["Enroll, Male: Avg. ACT Math Score (Enrl M)"].round(0).dropna(), return_counts=True)
    mCounts = mCounts/np.sum(mCounts)
    
    # New figure for Female data.
    p = figure(x_range = [0,36.5], plot_height = 500, plot_width = 500, title = " Average Score Distribution - Female")
    fBins = p.vbar(x = fVal, top = fCounts, width = 0.9)
    p.y_range.start = 0
    # Generate a line for the distribution of scores that year for Females
    x = np.linspace(0,36,500)
    fNorm = p.line(x, norm.pdf(x, fMean, stdDev), line_width = 2, color = "red")

    # New figure for Male data.
    q = figure(x_range = [0,36.5], plot_height = 500, plot_width = 500, title = " Average Score Distribution - Male")
    mBins = q.vbar(x = mVal, top = mCounts, width = 0.9)
    q.y_range.start = 0
    # Generate a line for the distribution of scores that year for Males
    mNorm = q.line(x, norm.pdf(x, mMean, stdDev), line_width = 2, color = "red")

    grid = layout([p], [q])

    t = show(grid, notebook_handle=True)
    t
    

    push_notebook(handle = t)



In [11]:
# Update is called here. This generates the notebook widget to change the values for the graphs above.
interact(update, Year=["2007","2008","2009","2010","2011","2012","2013","2014","2015","2016"], __manual=True)

interactive(children=(Dropdown(description='Year', options=('2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'), value='2007'), Output()), _dom_classes=('widget-interact',))

<function __main__.update>